In [1]:
import pandas as pd
import gzip
from tqdm import tqdm

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in tqdm(parse(path)):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
# GET THE DATA 
df = getDF('reviews_Amazon_Instant_Video_5.json.gz')

37126it [00:02, 16785.24it/s]


We start with the Amazon_Instant_Video (5-core) data because it is the smallest 
we can put them in a pandas dataframe 

In [2]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A11N155CW1UV02,B000H00VBQ,AdrianaM,"[0, 0]",I had big expectations because I love English ...,2.0,A little bit boring for me,1399075200,"05 3, 2014"
1,A3BC8O2KCL29V2,B000H00VBQ,Carol T,"[0, 0]",I highly recommend this series. It is a must f...,5.0,Excellent Grown Up TV,1346630400,"09 3, 2012"
2,A60D5HQFOTSOM,B000H00VBQ,"Daniel Cooper ""dancoopermedia""","[0, 1]",This one is a real snoozer. Don't believe anyt...,1.0,Way too boring for me,1381881600,"10 16, 2013"
3,A1RJPIGRSNX4PW,B000H00VBQ,"J. Kaplan ""JJ""","[0, 0]",Mysteries are interesting. The tension betwee...,4.0,Robson Green is mesmerizing,1383091200,"10 30, 2013"
4,A16XRPF40679KG,B000H00VBQ,Michael Dobey,"[1, 1]","This show always is excellent, as far as briti...",5.0,Robson green and great writing,1234310400,"02 11, 2009"


In [3]:
from datetime import datetime
from datetime import date, time
from dateutil.parser import parse
import numpy as np
import pandas as pd


/Users/sonia/anaconda/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


Populating the interactive namespace from numpy and matplotlib


/Users/sonia/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
#the dates are initially strings --> convert them to datetime format 
df.reviewTime=pd.to_datetime(df.reviewTime)

In [5]:
#extract the year alone
df['year']=list(map(lambda x: x.year, df['reviewTime']))
#df['month']=list(map(lambda x: x.month, df['reviewTime']))
#df['day']=list(map(lambda x: x.day, df['reviewTime']))
#df['date']=list(map(lambda x:x.strftime('%m-%d'), df['reviewTime']))

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,year,date
0,A11N155CW1UV02,B000H00VBQ,AdrianaM,"[0, 0]",I had big expectations because I love English ...,2.0,A little bit boring for me,1399075200,2014-05-03,2014,05-03
1,A3BC8O2KCL29V2,B000H00VBQ,Carol T,"[0, 0]",I highly recommend this series. It is a must f...,5.0,Excellent Grown Up TV,1346630400,2012-09-03,2012,09-03
2,A60D5HQFOTSOM,B000H00VBQ,"Daniel Cooper ""dancoopermedia""","[0, 1]",This one is a real snoozer. Don't believe anyt...,1.0,Way too boring for me,1381881600,2013-10-16,2013,10-16
3,A1RJPIGRSNX4PW,B000H00VBQ,"J. Kaplan ""JJ""","[0, 0]",Mysteries are interesting. The tension betwee...,4.0,Robson Green is mesmerizing,1383091200,2013-10-30,2013,10-30
4,A16XRPF40679KG,B000H00VBQ,Michael Dobey,"[1, 1]","This show always is excellent, as far as briti...",5.0,Robson green and great writing,1234310400,2009-02-11,2009,02-11


In [138]:
#create a timeline to view the number of reviews for each year : 
#we initialize the dataframe with all the days of the year : 
#(we choose 2008 because it is bisextile , but the year itself doesn't count)
Count=pd.DataFrame(pd.date_range('01-01-2008','31-12-2008'),columns=['dates'])
Count['dates']=list(map(lambda x:x.strftime('%m-%d'), Count['dates']))

In [132]:
def yearly_count (Y,Count):
    data = df[df.year==Y].groupby('reviewTime').count()
    frequency=data[['asin']].copy()
    frequency.columns=['year'+str(Y)]
    frequency['dates']=list(map(lambda x:x.strftime('%m-%d'), frequency.index))
    Count=pd.merge(Count,frequency,'outer')
    Count=Count.fillna(0)
    return Count


In [139]:
Years = sort(df.year.unique())
for y in Years:
    Count=yearly_count(y,Count)
#same comment as above for 2008
Count.dates=pd.to_datetime('2008-'+Count.dates)
Count.head()

In [173]:
#monthly average number of reviews for each year :
Count['month']=list(map(lambda x: x.month, Count['dates']))
month_average=Count.groupby('month').median()
month_average['time']=list(map(lambda x: pd.to_datetime('15-'+str(x)+'-2008'), month_average.index.values))

month_average

,year2000,year2004,year2006,year2007,year2008,year2009,year2010,year2011,year2012,year2013,year2014,time
month,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,21.0,70.0,2008-01-15
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,27.0,124.0,2008-02-15
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,4.0,27.0,86.0,2008-03-15
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,36.0,56.5,2008-04-15
5,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,5.0,35.0,55.0,2008-05-15
6,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,4.0,29.5,50.5,2008-06-15
7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,6.0,36.0,49.0,2008-07-15
8,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,7.0,37.0,0.0,2008-08-15
9,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,8.5,43.0,0.0,2008-09-15


In [175]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import DatetimeTickFormatter

# output to static HTML file
output_file("Videos_Number.html", title="Number of reviews")

# create a new plot with a a datetime axis type
p = figure(width=1000, height=500, x_axis_type="datetime")

# add renderers
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
for k in range(5,len(Years)):
    y=Years[k]
    col='year'+str(y)
    p.circle(Count.dates,Count[col], size=4,color=colors[-k], alpha=0.8,legend=str(y))
# NEW: customize by setting attributes
p.title.text = "Number of reviews by day"
p.legend.location = "top_left"
p.grid.grid_line_alpha=0
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of reviews'
p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d',years='',months='%B')
# show the results
show(p)

In [176]:
# output to static HTML file
output_file("Videos_Average.html", title="Average number of reviews ")

# create a new plot with a a datetime axis type
p = figure(width=1000, height=500, x_axis_type="datetime")

# add renderers
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
for k in range(5,len(Years)):
    y=Years[k]
    col='year'+str(y)
    p.line(month_average['time'],month_average[col], color=colors[-k],legend=str(y))

# NEW: customize by setting attributes
p.title.text = "Average Number of reviews by month"
p.legend.location = "top_left"
p.grid.grid_line_alpha=0
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Average Number of reviews'
p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d',years='',months='%B')
# show the results
show(p)

- make it more interactive 
- choose the year 
- compute the average/max/min over all the years 
- hover and view the value 
- why does it take so long ?!

In [187]:
# through the whole timeline 
evolution = df.groupby('reviewTime').count()

# output to static HTML file
output_file("Videos_evolution.html", title="Evolution of the number of reviews through the years")

# create a new plot with a a datetime axis type
p = figure(width=1000, height=500, x_axis_type="datetime")

# add renderers
avg=evolution.asin.rolling(182,center=True).mean() # moving average to have something smoother : is it any good here?
p.line(evolution.index.values,evolution.asin, color='navy')
p.line(evolution.index.values,avg, color='red')
# NEW: customize by setting attributes
p.title.text = "Daily Number of Reviews"
p.legend.location = "top_left"
p.grid.grid_line_alpha=0
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of reviews'
p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d',years='%Y',months='%B')
# show the results
show(p)

In [268]:
# set the time as an index : not necessarily a good idea 
df_time=df.set_index('reviewTime')
df_time=df_time.loc[df_time.index.sort_values()]


In [275]:
#other values than the number of review for each date :number of reviewers (up to that year)? length of reviews ? length of summary ?
#takes a very long time (too long -> think of something else to do this)
df['length_review']=list(map(lambda x:len(x),df.reviewText))
df['length_title_reviews']=list(map(lambda x:len(x),df.summary))
df['number of reviewers']=list(map(lambda x:len(df[df.year<=x].reviewerID.unique()),df.year))
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,year,date,length_review,length_title_reviews,number of reviewers
0,A11N155CW1UV02,B000H00VBQ,AdrianaM,"[0, 0]",I had big expectations because I love English ...,2.0,A little bit boring for me,1399075200,2014-05-03,2014,05-03,159,26,"[A11N155CW1UV02, A3BC8O2KCL29V2, A60D5HQFOTSOM..."
1,A3BC8O2KCL29V2,B000H00VBQ,Carol T,"[0, 0]",I highly recommend this series. It is a must f...,5.0,Excellent Grown Up TV,1346630400,2012-09-03,2012,09-03,186,21,"[A3BC8O2KCL29V2, A16XRPF40679KG, A1POFVVXUZR3I..."
2,A60D5HQFOTSOM,B000H00VBQ,"Daniel Cooper ""dancoopermedia""","[0, 1]",This one is a real snoozer. Don't believe anyt...,1.0,Way too boring for me,1381881600,2013-10-16,2013,10-16,134,21,"[A3BC8O2KCL29V2, A60D5HQFOTSOM, A1RJPIGRSNX4PW..."
3,A1RJPIGRSNX4PW,B000H00VBQ,"J. Kaplan ""JJ""","[0, 0]",Mysteries are interesting. The tension betwee...,4.0,Robson Green is mesmerizing,1383091200,2013-10-30,2013,10-30,141,27,"[A3BC8O2KCL29V2, A60D5HQFOTSOM, A1RJPIGRSNX4PW..."
4,A16XRPF40679KG,B000H00VBQ,Michael Dobey,"[1, 1]","This show always is excellent, as far as briti...",5.0,Robson green and great writing,1234310400,2009-02-11,2009,02-11,1300,30,"[A16XRPF40679KG, A2QWF2BJ2FM4R2, ATECU14VUIOJ0..."
5,A1POFVVXUZR3IQ,B000H00VBQ,Z Hayes,"[12, 12]",I discovered this series quite by accident. Ha...,5.0,I purchased the series via streaming and loved...,1318291200,2011-10-11,2011,10-11,3039,50,"[A16XRPF40679KG, A1POFVVXUZR3IQ, A356RFKNIG043..."
6,A1PG2VV4W1WRPL,B000H0X79O,"Jimmy C. Saunders ""Papa Smurf""","[0, 0]","It beats watching a blank screen. However, I j...",3.0,It takes up your time.,1381795200,2013-10-15,2013,10-15,99,22,"[A3BC8O2KCL29V2, A60D5HQFOTSOM, A1RJPIGRSNX4PW..."
7,ATASGS8HZHGIB,B000H0X79O,JohnnyC,"[0, 0]","There are many episodes in this series, so I p...",3.0,A reasonable way to kill a few minutes,1388275200,2013-12-29,2013,12-29,348,38,"[A3BC8O2KCL29V2, A60D5HQFOTSOM, A1RJPIGRSNX4PW..."
8,A3RXD7Z44T9DHW,B000H0X79O,Kansas,"[0, 0]",This is the best of the best comedy Stand-up. ...,5.0,kansas001,1393372800,2014-02-26,2014,02-26,186,9,"[A11N155CW1UV02, A3BC8O2KCL29V2, A60D5HQFOTSOM..."
9,AUX8EUBNTHIIU,B000H0X79O,Louis V. Borsellino,"[0, 0]",Not bad. Didn't know any of the comedians but...,3.0,Entertaining Comedy,1396396800,2014-04-02,2014,04-02,127,19,"[A11N155CW1UV02, A3BC8O2KCL29V2, A60D5HQFOTSOM..."


In [279]:
#Now we see the number of reviews for each product 
products = df.groupby('asin').count()
products=products[['reviewerID']]
products

,reviewerID
asin,
B000H00VBQ,6
B000H0X79O,5
B000H29TXU,5
B000H2DMME,10
B000H4YNM0,24
B000HAB4NK,33
B000HKWE3O,5
B000HZEHL6,22
B000I5PVD8,10


In [243]:
#We can isolate a product by his ID:
product = df[df.asin=='B00L86ZKAK'].copy()
product=product.sort_values('reviewTime')
product.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,year,date
36999,A3KV5PN9P44YG0,B00L86ZKAK,Melanie,"[2, 4]",One of my fav shows,5.0,Five Stars,1404345600,2014-07-03,2014,07-03
36976,AJT9NDFFCC5M9,B00L86ZKAK,gabbygirl,"[2, 3]",Different from most TV and a great escape from...,4.0,Magnetic Magic,1404691200,2014-07-07,2014,07-07
36957,A1X321SRVI41S3,B00L86ZKAK,Angel Mom,"[2, 4]","Love it. The suspense the drama, the cliff han...",5.0,Love it. The suspense the drama,1404691200,2014-07-07,2014,07-07
36959,A17WNTA00WZ65L,B00L86ZKAK,Betsy Bree,"[4, 7]",Even though the first episode was written by S...,2.0,Silly Stuff,1404691200,2014-07-07,2014,07-07
37006,A18HE80910BTZI,B00L86ZKAK,"rbhatta ""A Dinosaur you can trust!""","[1, 2]",Did the producer have tough time keeping the o...,2.0,Why kill old stars?,1404691200,2014-07-07,2014,07-07


In [240]:
#for each product,we define new variables of interest:
#some are stored in a dictionary 
info={'1-id_product':product.asin.unique()[0]}
info['3-rating_average'] = product.overall.mean()
info['2-number_reviewers'] =len(product.reviewerID.unique())
#number of positive reviews
#number of negative reviews
#average sentiment score

#some are added to the dataframe 
product['length_reviews']=list(map(lambda x:len(x),product.reviewText))
product['length_title_reviews']=list(map(lambda x:len(x),product.summary))
info

{'1-id_product': 'B00L86ZKAK',
 '2-number_reviewers': 71,
 '3-rating_avg': 3.619718309859155}

In [250]:
#view any variable of interest on a timeline 
#(we need to compute the average in case we have many observations for the same day)
def product_timeline (variable):
    p = figure(width=1000, height=500, x_axis_type="datetime")
    avg=product.groupby('reviewTime').mean() # We compute the mean for each date 
    p.line(avg.index.values,avg[variable], color='navy')
    p.title.text = "Product Reviews"
    p.xaxis.axis_label = 'Date'
    show(p)

In [251]:
product_timeline('overall')